In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [11]:
import numpy as np
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import os,sys
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, f1_score

from helpers import *

## Loading the Original Training Set For Linear Modelling

In [13]:
image_path = str(Path.cwd().parent) + "/data/original/images/"
files = os.listdir(image_path)
n = len(files) 
print("Loading " + str(n) + " images")
imgs = [load_image(image_path + files[i]) for i in range(n)]
print(files[0])

Loading 100 images


NameError: name 'mpimg' is not defined

In [ ]:
gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = []
for i in range (n):
    img = mpimg.imread(gt_dir + files[i])
    try:
        gt_imgs.append(img[:,:,0])
    except:
        gt_imgs.append(img)
print(files[0])

Loading 100 images
satImage_091.png


In [ ]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [ ]:
# Extract 6-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(feat_m, feat_v)
    return feat

# Extract 2-dimensional features consisting of average gray color as well as variance
def extract_features_2d(img):
    feat_m = np.mean(img)
    feat_v = np.var(img)
    feat = np.append(feat_m, feat_v)
    return feat

# Extract features for a given image
def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
    return X

In [ ]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

X = np.asarray([extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

## Linear Modelling Using Logistic Regression and SVM

In [ ]:
logreg = LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(x_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced')

In [ ]:
y_hat = logreg.predict(x_test)
f1 = f1_score(y_test, y_hat)

print(f'F1-Score: {f1}')

F1-Score: 0.4503493750615096


In [ ]:
svm = LinearSVC()
svm.fit(x_train, y_train)

LinearSVC()

In [ ]:
y_hat = svm.predict(x_test)
f1 = f1_score(y_test, y_hat)
print(f'F1-Score: {f1}')

F1-Score: 0.0


## Loading the Augmented Training Set for Augmented Modelling

In [ ]:
root_dir = "training_augmented/"
# Loading the images
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) 
imgs = [load_image(image_dir + files[i]) for i in range(n)]

# Loading the groundtruth images
gt_dir = root_dir + "groundtruth/"
gt_imgs = []
for i in range (n):
    img = mpimg.imread(gt_dir + files[i])
    try:
        gt_imgs.append(img[:,:,0])
    except:
        gt_imgs.append(img)

In [ ]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [ ]:
foreground_threshold = 0.25
X = np.asarray([extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

## Linear Modelling Using SVM and Logistic Regression

In [ ]:
logreg = LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(x_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced')

In [ ]:
y_hat = logreg.predict(x_test)
f1 = f1_score(y_test, y_hat)

print(f'F1-Score: {f1}')

F1-Score: 0.40000676620506115


In [ ]:
svm = LinearSVC()
svm.fit(x_train, y_train)

LinearSVC()

In [ ]:
y_hat = svm.predict(x_test)
f1 = f1_score(y_test, y_hat)
print(f'F1-Score: {f1}')

F1-Score: 0.0


In [ ]:
svm = SVC()
svm.fit(x_train, y_train)
y_hat = svm.predict(x_test)
f1 = f1_score(y_test, y_hat)
print(f'F1-Score: {f1}')